## Web Crawling in i-SMART
2018년 02월 14일  
mosuklee@gmail.com

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
user_id = "0140398763"
user_pw = "skt0119"

#15분 DATA 분석기간
search_year = 2017
start_month = 1
end_month = 3

In [3]:
# login 하고자 하는 login page를 url에 저장한다. 네이버는 login page가 https://nid.naver.com/nidlogin.login에 있다.
url = "https://pccs.kepco.co.kr/iSmart/jsp/cm/login/main.jsp"

In [4]:
#C rome driver를 가상브라우져에 뛰우기 -----(1)
browser = webdriver.Chrome ("C:/webdriver/chromedriver")

# 3초 대기하기 ----(2)
browser.implicitly_wait(3)

In [5]:
# 로그인하기   ---- (3)
browser.get(url)
element_id = browser.find_element_by_name('userId')
element_id.clear()
element_id.send_keys(user_id)
element_pw = browser.find_element_by_name('password')
element_pw.clear()
element_pw.send_keys(user_pw)

#버튼누르기
#  //*[@id="frmNIDLogin"]/fieldset/input
#  //*[@id="contents_main"]/div/form/div[1]/div[2]/input
browser.find_element_by_xpath('//*[@id="contents_main"]/div/form/div[1]/div[2]/input').click()

#print(browser)

#browser.save_screenshot("website_A.png")

In [6]:
# Naver 페이 들어가기
browser.get('https://pccs.kepco.co.kr/iSmart/cm/login/readEquip.do?flag=AB')
#browser.get('https://pccs.kepco.co.kr/iSmart/pccs/usage/readTrendMonthList.do?month=12&year=2017')
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
#notices = soup.select('p.user_infor > strong')
#print(notices)
items=soup.find_all("td",class_="al_left")
#for item in items:
#    print(item)

################################################################################################
# 딕션어리를 이용하여 리스트로 Add한 다음 dataframe을 만들고 이를 csv로 저장하는 모듈을 만든다 #
################################################################################################

# 1, 데이터 변수 선언
l=[] #리스트 선언
d={} #딕션어리 선언

# 2 Crawling한 자료를 딕션어리로 만들어 하나씩 저장한다.
d["계약자"]=items[10].text.strip()
d["계약전력"]=items[0].text.strip()
d["계약종별"]=items[1].text.strip()
d["공급방식"]=items[2].text.strip()
d["용    도"]=items[5].text.strip()
d["분석년도"]=search_year
d["시작월"]=start_month
d["종료월"]=end_month

# 3. 딕션어리로 만든 Crawling Data를 List에 Append시킨다.
l.append(d) 

# 4. List로 만든 파일을 Pandas를 사용하여 데이터프레임으로 만든다.
df = pd.DataFrame(l) # pandas 사용 l의 데이터프레임화

# 5. 데이터프레임으로 만든 파일을 csv파일로 저장한다.
outputfile ='%s_userinfo_ismart.csv'%(user_id)
df.to_csv(outputfile, encoding='UTF-8', index=False)
print("Success Get DataFIle and Save Data")

#################################################################################################

print("[고객정보]")
print("계 약 자 :",items[10].text.strip())
print("계약전력:",items[0].text.strip())
print("계약종별:",items[1].text.strip())
print("공급방식:",items[2].text.strip())
print("용    도:",items[5].text.strip())
print("분석년도:",search_year)
print("시작월 :",start_month)
print("종료월 :",end_month)


Success Get DataFIle and Save Data
[고객정보]
계 약 자 : 0140398969(에스케이텔레콤（주）)
계약전력: 4750 kW
계약종별: 일반용(을)고압A
공급방식: 특고압3￠4W22.9Kv-y
용    도: 상업용
분석년도: 2017
시작월 : 1
종료월 : 3


In [7]:
browser.get("https://pccs.kepco.co.kr/iSmart/pccs/amr/readCharge.do?select=1&sel=2017")
html = browser.page_source
data = pd.read_html(html,header=0)
df = pd.DataFrame(data[0]) # pandas 사용 l의 데이터프레임화
df.columns = ["year","con_kw","price_kw","use_kw","use_day","pf1","pf2","elec_price"]
head_alias = ["년도","계약전력(kW)","요금적용전력(kW)","사용전력량(kW)","사용일수","지상(%)","지상(%)","전기요금(원)"]
df=df.drop(df.index[0])  # Dataframe의 첫번째 Row를 삭제한다.
outputfile ='%s_elecdata_ismart.csv'%(user_id)
df.to_csv(outputfile, encoding='UTF-8', index=False)
print("Success Get DataFIle and Save Data")

Success Get DataFIle and Save Data


In [9]:
import urllib.parse

frames1 = []
frame_15=[]
month_day = [31,28,31,30,31,30,31,31,30,31,30,31]
i = 1
#frames1 = pd.DataFrame(frames1)

for i in range(start_month,end_month+1):
      for search_day in range (1,month_day[i-1]+1):
            url = "https://pccs.kepco.co.kr/iSmart/pccs/usage/getGlobalUsageStats.do" 
            values = {
                "day": str(search_day),
                "diodGubun": "0",
                "diodval": "30",
                "month": str(i),
                "reviseFlag": "30",
                "searchType_min": "15",
                "year" : str(search_year)
                }
            params =  urllib.parse.urlencode(values)
            url = url + "?"+params
            browser.get(url)
            html = browser.page_source
            #가상브라우져에서 해당  Html파일을 추출한다.
            data_15 = pd.read_html(html,header=0)
            #표가 반으로 나누어진 것을 합친다.
            frame_15 = data_15[3]
            frame_15 = frame_15.append(data_15[4])
            frame_15['시'] = str(search_year)+"."+str(i)+"."+str(search_day)+"."+frame_15['시']
            if i == 1 :
                frames1 = frame_15
                i=2
            else :
                frames1=frames1.append(frame_15)
        
frames1 = pd.DataFrame(frames1)
#print(frames1)
#df_15 = pd.concat(frames1)
df_15 = frames1
df_15.columns = ["hr","use_kwh","max_kw","sisang","jinsang","co2","pf1","pf2"]
#head_alias = ["년도","계약전력(kW)","요금적용전력(kW)","사용전력량(kW)","사용일수","지상(%)","지상(%)","전기요금(원)"]
df_15 = df_15.drop(df_15.index[0])  # Dataframe의 첫번째 Row를 삭제한다.
outputfile ='%s_elecdata_15_ismart.csv'%(user_id)
df_15.to_csv(outputfile, encoding='UTF-8', index=False)
print("Success Get DataFIle and Save Data")

Success Get DataFIle and Save Data


In [10]:
browser.quit()